In [1]:
import utils.dataset_utils as dataset
import utils.train_utils as train
import preprocessing.GSP as GSP
import numpy as np
import copy

In [2]:
filenames = ["../data/HSI/Liver_map_150z25_60s_1TCPOBOP.npy",
             "../data/HSI/Liver_map_150z25_60s_2TCPOBOP.npy",
             "../data/HSI/Liver_map_150z25_60s_3OBOB.npy"]
data = []
for f in filenames:
    data.append(np.load(f))

# assume linear increase in wavelength 
wavelength = list(range(data[0].shape[2]))
shape = data[0].shape

# raman_data = []
# photo_data = []
# for x in data:
#     tmp = copy.copy(x.reshape(-1, x.shape[-1]))
#     tmp1, tmp2 = GSP.split_Raman_af(GSP.smoothing(tmp), wavelength)
#     raman_data.append(tmp1)
#     photo_data.append(tmp2)

In [4]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

BATCH_SIZE = 64
EPOCHS = 10
PATCH_SIZE = 5
data = np.array(data)
train_loader, test_loader = dataset.load_liver(data, BATCH_SIZE, PATCH_SIZE)

for batch_idx, (x, *_) in enumerate(train_loader):
    print(batch_idx, x.shape)

0 torch.Size([64, 1, 5, 5, 1000])
1 torch.Size([64, 1, 5, 5, 1000])
2 torch.Size([64, 1, 5, 5, 1000])
3 torch.Size([64, 1, 5, 5, 1000])
4 torch.Size([64, 1, 5, 5, 1000])
5 torch.Size([64, 1, 5, 5, 1000])
6 torch.Size([64, 1, 5, 5, 1000])
7 torch.Size([64, 1, 5, 5, 1000])
8 torch.Size([64, 1, 5, 5, 1000])
9 torch.Size([64, 1, 5, 5, 1000])
10 torch.Size([64, 1, 5, 5, 1000])
11 torch.Size([64, 1, 5, 5, 1000])
12 torch.Size([64, 1, 5, 5, 1000])
13 torch.Size([64, 1, 5, 5, 1000])
14 torch.Size([64, 1, 5, 5, 1000])
15 torch.Size([64, 1, 5, 5, 1000])
16 torch.Size([64, 1, 5, 5, 1000])
17 torch.Size([64, 1, 5, 5, 1000])
18 torch.Size([64, 1, 5, 5, 1000])
19 torch.Size([64, 1, 5, 5, 1000])
20 torch.Size([64, 1, 5, 5, 1000])
21 torch.Size([64, 1, 5, 5, 1000])
22 torch.Size([64, 1, 5, 5, 1000])
23 torch.Size([64, 1, 5, 5, 1000])
24 torch.Size([64, 1, 5, 5, 1000])
25 torch.Size([64, 1, 5, 5, 1000])
26 torch.Size([64, 1, 5, 5, 1000])
27 torch.Size([64, 1, 5, 5, 1000])
28 torch.Size([64, 1, 5, 5, 10

In [ ]:
class SupervisedClassifier(BaseEstimator):
    def __init__(self, **kwargs):
        self.kwargs = {}
        self.ae_kwargs = {}
        self.k_means_kwargs = {}                
        self.set_params(**kwargs)

        _use_cuda = torch.cuda.is_available() and kwargs['cuda']
        if _use_cuda:
            torch.backends.cudnn.enabled = True
            torch.backends.cudnn.benchmark = True
        self.device = torch.device('cuda' if _use_cuda else 'cpu')        
        
    def fit(self, x, **kwargs):
        self.set_params(**kwargs)
        X = unit_vector_norm(x)
        
        ###################### Autoencoder ################################
        self.model = AutoEncoder(**self.ae_kwargs).to(self.device)
        
        parameters = filter(lambda x: x.requires_grad, self.model.parameters())
        self.optimizer = optim.Adam(parameters)        
        train_loader, test_loader = dataset.load_liver(X, self.kwargs['batch_size'])
        
        for epoch in range(self.kwargs['epochs']):
            print('-'*50)
            print('Epoch {:3d}/{:3d}'.format(epoch+1, self.kwargs['epochs']))
            start_time = datetime.now()
            train.train(self.model, self.optimizer, train_loader, self.kwargs['loss_func'], self.kwargs['log_step'], self.device)
            end_time = datetime.now()
            time_diff = relativedelta(end_time, start_time)
            print('Elapsed time: {}h {}m {}s'.format(time_diff.hours, time_diff.minutes, time_diff.seconds))
            loss = train.test(self.model, test_loader, self.kwargs['loss_func'], self.device)
            print('Validation| bits: {:2.2f}'.format(loss), flush=True)    
          
        self.model.eval()
        with torch.no_grad():
            W = self.model.encode(dataset.load_liver_all(X).to(self.device))
        self.z = W
        W = W.cpu().detach().numpy()
        
        ###################### clustering ################################        
        self.clusters = cluster.KMeans(**self.k_means_kwargs).fit(W)
        self.clusters = self.clusters.labels_
        
        one_hot = np.zeros((X.shape[0], self.kwargs['n_clusters']), dtype=bool)
        one_hot[range(X.shape[0]), self.clusters] = 1
                          
        ###################### reference spectra ################################
        self.reference_spectra_ = unit_vector_norm(np.array([np.abs(x_) for i, x_ in enumerate(one_hot.T @ X)]))
        self.ref_org = np.array([x[one_hot[:,i],:].mean(axis=0) for i in range(self.kwargs['n_clusters'])])
               
        # Return the classifier
        return self
    
    def predict(self, X):
        """
        predict transforms the data into the reference space. Min weight should be 0 or higher then 'min_weight'
        The error is the NMSE, where the MSE is normalised by the signal strength. 
        error.shape = X.shape[0], so for each data point the error is calculated.
        """
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)
        X = unit_vector_norm(X)
        
        ###################### RCA ################################           
        RCA_vector = np.array([optimize.nnls(self.reference_spectra_.T, X[i,:])[0] for i in range(X.shape[0])])

        return RCA_vector
    
    def get_reference_vectors(self):
        return self.reference_spectra_

    def get_org_reference_vectors(self):
        return self.ref_org    
    
    def get_params(self, deep=False):
        return self.kwargs
    
    def set_params(self, **kwargs):
        self.kwargs.update(kwargs)
        self.k_means_kwargs.update({k:v  for k,v in kwargs.items() if k in list(inspect.signature(cluster.KMeans).parameters.keys())})     
        self.ae_kwargs.update({k:v  for k,v in kwargs.items() if k in list(inspect.signature(AutoEncoder).parameters.keys())})     
        return self